In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# Evaluating CNN+RNN models on the dataset

# Imports
import gc
import os, random
from collections import Counter
from datetime import datetime

import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from wandb.keras import WandbCallback

import wandb
from wp8.pre_processing.utils import safe_mkdir
from wp8.utils.cnn_rnn_utils import load_and_split
from wp8.utils.dataset import TimeSeriesGenerator as TSG

In [7]:
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)


In [8]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""

    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


opt = dotdict(
    {
        "lstm1_units": 128,
        "lstm2_units": 64,
        "dense_units": 32,
        "dropout": 0.5,
        "epochs": 1000,
        "train_actors": [1],
        "val_actors": [2],
        "train_cams": [1],
        "val_cams": [1],
        "seq_len": 20,
        "split_ratio": None,
        "drop_offair": False,
        "undersample": False,
        "batch_size": 40,
        "stride": 10,
        "learning_rate": 1e-4,
        "micro_classes": False,
    }
)


In [9]:
X_train, y_train, X_val, y_val, cams_train, cams_val = load_and_split(
    opt.train_actors, opt.val_actors, opt.train_cams, opt.val_cams, opt.split_ratio, opt.drop_offair, opt.undersample, opt.micro_classes
)
print(f"\nX_train shape: {X_train.shape}, len y_train: {len(y_train)}, X_val shape: {X_val.shape}, len y_val: {len(y_val)}\n")


[STATUS] Load Train Set


Loading features: outputs/dataset/features/Actor_1_Walk_Stick_Full_PH.npz: 100%|██████████| 16/16 [00:19<00:00,  1.24s/it]
Loading csv datasets: 100%|██████████| 16/16 [00:02<00:00,  6.17it/s]


[STATUS] Load Val Set


Loading features: outputs/dataset/features/Actor_2_Walk_Stick_Full_PH.npz: 100%|██████████| 19/19 [00:17<00:00,  1.07it/s]
Loading csv datasets: 100%|██████████| 19/19 [00:00<00:00, 51.25it/s]



X_train shape: (60480, 2048), len y_train: 60480, X_val shape: (57120, 2048), len y_val: 57120



In [10]:
series_gen = TSG(opt)
X_train_series, y_train_series, class_weights, classes = series_gen.get_train_series(X_train, y_train, cams_train)
X_val_series, y_val_series = series_gen.get_val_series(X_val, y_val, cams_val)
print(f"\nX_train_series shape: {X_train_series.shape}, len y_train_series: {len(y_train_series)}, X_val_series shape: {X_val_series.shape}, len y_val_series: {len(y_val_series)}\n")


100%|██████████| 4536/4536 [00:00<00:00, 169867.80it/s]


Classes mapping:
{'adl': 1, 'falling': 2, 'lying_down': 3}

Class weights for train series:
[0.47772512 4.99009901 1.41573034]


100%|██████████| 4284/4284 [00:00<00:00, 180053.09it/s]



X_train_series shape: (4536, 20, 2048), len y_train_series: 4536, X_val_series shape: (4284, 20, 2048), len y_val_series: 4284



In [11]:
print(f"Before resampling - y_train_series shape: {Counter(y_train_series)}")

Before resampling - y_train_series shape: Counter({0: 3165, 2: 1068, 1: 303})


### Resampling


In [12]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline
from imblearn.combine import SMOTETomek
import copy

undersampling_strategy = {0: 3000, 2: 3000, 1: 3000}
oversampling_strategy = {0: 23890, 2: 6437, 1: 4000}
under = RandomUnderSampler(random_state=2, sampling_strategy=undersampling_strategy)
over = RandomOverSampler(random_state=2, sampling_strategy=oversampling_strategy)
steps = [("o", over), ("u", under)]
pipeline = Pipeline(steps=steps)
# smt = SMOTETomek(random_state=2)


In [13]:
print(f"Original X_train_series shape: {X_train_series.shape}")

X_train_series = np.reshape(X_train_series, (-1, 20 * 2048))
print(f"X_train_series reshaped: {X_train_series.shape}")

X_train_series, y_train_series = pipeline.fit_resample(X_train_series, y_train_series)

X_train_series = np.reshape(X_train_series, (-1, 20, 2048))

print(f"After resampling X_train_series shape: {X_train_series.shape}")
print(f"After resampling - y_train_series count: {Counter(y_train_series)}")


Original X_train_series shape: (4536, 20, 2048)
X_train_series reshaped: (4536, 40960)


/Users/andrea/miniforge3/envs/ml/lib/python3.9/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (23890) in class 0 will be larger than the number of samples in the majority class (class #0 -> 3165)
  warnings.warn(
/Users/andrea/miniforge3/envs/ml/lib/python3.9/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (6437) in class 2 will be larger than the number of samples in the majority class (class #0 -> 3165)
  warnings.warn(
/Users/andrea/miniforge3/envs/ml/lib/python3.9/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (4000) in class 1 will be larger than the number of samples in the majority class (class #0 -> 3165)
  warnings.warn(


After resampling X_train_series shape: (9000, 20, 2048)
After resampling - y_train_series count: Counter({0: 3000, 1: 3000, 2: 3000})


In [14]:
# WANDB project initialization
run = wandb.init(
    project="Fall detection CNN + RNN",
    config={
        "model": "LSTM",
        "epochs": opt.epochs,
        "seq_len": opt.seq_len,
        "num_features": 2048,
        "batch_size": opt.batch_size,
        "stride": opt.stride,
        "loss_function": "sparse_categorical_crossentropy",
        "architecture": "LSTM",
        "train_actors": opt.train_actors,
        "val_actors": opt.val_actors,
        "train_cams": opt.train_cams,
        "val_cams": opt.val_cams,
        "micro_classes": opt.classes,
        "dropout": opt.dropout,
        "lstm1_units": opt.lstm1_units,
        "lstm2_units": opt.lstm2_units,
        "dense_units": opt.dense_units,
        "learning_rate": opt.learning_rate,
        "split_ratio": opt.split_ratio,
        "drop_offair": opt.drop_offair,
        "undersample": opt.undersample,
    },
)

cfg = wandb.config


wandb: Currently logged in as: andreaapi. Use `wandb login --relogin` to force relogin


In [15]:
model = Sequential()
model.add(LSTM(units=cfg.lstm1_units, input_shape=(cfg.seq_len, cfg.num_features), return_sequences=True))
model.add(Dropout(cfg.dropout))
model.add(LSTM(units=cfg.lstm2_units, input_shape=(cfg.seq_len, cfg.num_features), return_sequences=False))
model.add(Dropout(cfg.dropout))
model.add(Dense(units=cfg.dense_units, activation="relu"))
model.add(Dropout(cfg.dropout))
model.add(Dense(units=np.unique(y_train_series, axis=0).shape[0], activation="softmax"))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=cfg.learning_rate),
    loss=cfg.loss_function,
    metrics=["accuracy"],
)
model.summary()


Metal device set to: Apple M1


2022-07-20 17:52:32.888313: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-20 17:52:32.889285: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 20, 128)           1114624   
                                                                 
 dropout (Dropout)           (None, 20, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 9

In [16]:
# Callbacks
dir_path = f"model_checkpoints/{cfg.model}"
safe_mkdir(dir_path)
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y_%H:%M:%S")

model_checkpoint = ModelCheckpoint(
    filepath=f"{dir_path}/{cfg.model}_{dt_string}",
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True,
    initial_value_threshold=0.8,
    verbose=1,
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=30,
    verbose=1,
    mode="min",
    min_delta=1e-5,
    cooldown=1,
    min_lr=1e-6,
)

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=20,
    verbose=1,
    mode="min",
)

callbacks = [WandbCallback(), reduce_lr, early_stop]


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [17]:
# Train Model
history = model.fit(X_train_series, y_train_series, validation_data=(X_val_series, y_val_series), epochs=cfg.epochs, callbacks=callbacks, batch_size=cfg.batch_size)


Epoch 1/1000


2022-07-20 17:52:52.096401: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-20 17:52:55.466836: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-20 17:52:55.978553: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-20 17:52:56.486865: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-20 17:52:57.423113: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-20 17:52:57.734301: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


224/225 [============================>.] - ETA: 0s - loss: 0.8997 - accuracy: 0.5417

2022-07-20 17:53:11.454673: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-20 17:53:11.670753: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-20 17:53:11.756692: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


225/225 [==============================] - 22s 71ms/step - loss: 0.8992 - accuracy: 0.5421 - val_loss: 0.9348 - val_accuracy: 0.6090 - _timestamp: 1658332393.0000 - _runtime: 51.0000 - lr: 1.0000e-04
Epoch 2/1000
225/225 [==============================] - 11s 47ms/step - loss: 0.6968 - accuracy: 0.6872 - val_loss: 0.9094 - val_accuracy: 0.5254 - _timestamp: 1658332404.0000 - _runtime: 62.0000 - lr: 1.0000e-04
Epoch 3/1000
225/225 [==============================] - 9s 40ms/step - loss: 0.5317 - accuracy: 0.8031 - val_loss: 0.8408 - val_accuracy: 0.6396 - _timestamp: 1658332413.0000 - _runtime: 71.0000 - lr: 1.0000e-04
Epoch 4/1000
225/225 [==============================] - 9s 40ms/step - loss: 0.4642 - accuracy: 0.8340 - val_loss: 0.8284 - val_accuracy: 0.6254 - _timestamp: 1658332422.0000 - _runtime: 80.0000 - lr: 1.0000e-04
Epoch 5/1000
225/225 [==============================] - 8s 35ms/step - loss: 0.4351 - accuracy: 0.8426 - val_loss: 0.7737 - val_accuracy: 0.6807 - _timestamp: 1658

In [18]:
# Evaluate Model
val_logits = model.predict(X_val_series, batch_size=60, verbose=1)


2022-07-20 17:56:58.117224: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-20 17:56:59.093105: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 2/72 [..............................] - ETA: 3s  

2022-07-20 17:57:01.730702: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


72/72 [==============================] - 6s 21ms/step


In [19]:
# free up memory
# del X_train_series
# del y_train_series
# del X_val_series
# del y_val_series

# gc.collect()


In [20]:
# Log metrics to wandb
y_pred_val_classes = np.argmax(val_logits, axis=1).tolist()

cr = classification_report(y_val_series, y_pred_val_classes, target_names=classes)

wandb.log({"Classification Report": cr})

wandb.sklearn.plot_roc(y_val_series, val_logits, classes)
wandb.sklearn.plot_class_proportions(y_train_series, y_val_series, classes)
wandb.sklearn.plot_precision_recall(y_val_series, val_logits, classes)
wandb.sklearn.plot_confusion_matrix(y_val_series, y_pred_val_classes, classes)
wandb.join()


wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.


accuracy,▁▄▆▆▆▇▇▇▇▇▇▇▇██▇█████████
epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,█▆▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁
val_accuracy,▄▁▅▄▆▆▇█▇█▇▆▇██▆▇▇▅█▆▆▆▄▄
val_loss,▃▃▂▂▁▂▂▂▂▂▂▃▃▄▄▃▆▅▅▄▇▆▆█▇
Classification Report,precis...
accuracy,0.92911
best_epoch,4
best_val_loss,0.77372
epoch,24
loss,0.19935


In [21]:
print(cr)

              precision    recall  f1-score   support

         adl       0.85      0.71      0.77      3244
     falling       0.09      0.23      0.13       327
  lying_down       0.44      0.45      0.45       713

    accuracy                           0.63      4284
   macro avg       0.46      0.46      0.45      4284
weighted avg       0.72      0.63      0.67      4284

